In [1]:
from sklearn import svm 
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

#tree
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from IPython.display import Image # displaying images files in jupyter
from IPython.display import IFrame # displaying pdf file in jupyter

import warnings
warnings.simplefilter("ignore")

In [22]:
#read the data
match = pd.read_csv("DataSetForClassification.csv")

#drop the columns that won't be used for prediction
match.drop("Unnamed: 0", axis = 1, inplace = True)
match.drop("Unnamed: 58", axis = 1, inplace = True)
match.drop(match.ix[:, "home_player_1": "away_player_11"], axis = 1, inplace = True)

#split the data into training and validation set
#season 2008-09 to 2014-15 will be the training data
#we will predict the outcome of the matches in 2015-16
train = match.loc[match["Season"] != "2015/2016",]
validation = match.loc[match["Season"] == "2015/2016",]

#trees
X = train.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y =  train['Outcome']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,Y)

#predict - try an out of sample prediction
X = validation.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y = validation['Outcome']

validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

1
0.997368421053


In [27]:
#random forest on train data
X = train.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y =  train['Outcome']
clf_rf = RandomForestClassifier(max_features="sqrt")
clf_rf = clf_rf.fit(X, Y)
clf_rf_scores = cross_val_score(clf_rf, X, Y, cv=5)

#try an out of sample prediction
X = validation.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y = validation['Outcome']
validation['prediction'] = clf_rf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

28
0.926315789474


In [28]:
#try boosting
X = train.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y =  train['Outcome']
clf = AdaBoostClassifier()
clf = clf.fit(X,Y)
clf.score(X,Y)

#try an out of sample prediction
X = validation.ix[:, "Home_Team_ID":"HomeVsAway_Away"]
Y = validation['Outcome']
validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

105
0.723684210526
